In [242]:
import numpy as np
from random import random
from queue import PriorityQueue, SimpleQueue
from functools import reduce
from tqdm import tqdm
from math import ceil

In [215]:
PROBLEM_SIZE = 20
NUM_SETS = 40

In [198]:
def random_sets():
    sets = tuple(sorted([np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)], key=lambda x: -sum(x)))
    while not goal_check((set(range(NUM_SETS)), set()), sets):
        sets = tuple(sorted([np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)], key=lambda x: -sum(x)))
    return sets

In [244]:
def goal_check(state, sets):
    return np.all(reduce(
        np.logical_or,
        [sets[i] for i in state[0]],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def h_not_admissible(state, sets):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [sets[i] for i in state[0]],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

def h(state, sets):
    already_covered = reduce(
            np.logical_or,
            [sets[i] for i in state[0]],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    if np.all(already_covered):
        return 0
    missing = PROBLEM_SIZE - sum( already_covered )
    largest_to_take = max(sum(np.logical_and(sets[set_id], np.logical_not(already_covered))) for set_id in state[1]) # take only the next possible sets
    return ceil(missing / largest_to_take)

def professorh(state, sets):
    already_covered = reduce(
            np.logical_or,
            [sets[i] for i in state[0]],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in sets), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def professorf(state, sets):
    return len(state[0]) + professorh(state, sets)

def g(state):
    return len(state[0])

def f(state, sets):
    return g(state) + h(state, sets)


In [237]:
def find_sol(sets):
    frontier = PriorityQueue()
    actions = filter(lambda x: sum(sets[x]) != 0, range(NUM_SETS))
    frontier.put( (0, (set(), set(actions))) )
    current_state = frontier.get()[1]
    counter = 0

    while not goal_check(current_state, sets):
        counter += 1
        for action in current_state[1]:
            if sum(sets[action]) == 0:
                continue
            new_state = (current_state[0] | {action}, current_state[1] - {action})
            frontier.put( (f(new_state, sets), new_state) )
        current_state = frontier.get()[1]
    return counter, len(current_state[0])

def find_sol_professor(sets):
    frontier = PriorityQueue()
    actions = filter(lambda x: sum(sets[x]) != 0, range(NUM_SETS))
    frontier.put( (0, (set(), set(actions))) )
    current_state = frontier.get()[1]
    counter = 0

    while not goal_check(current_state, sets):
        counter += 1
        for action in current_state[1]:
            new_state = (current_state[0] | {action}, current_state[1] - {action})
            frontier.put( (professorf(new_state, sets), new_state) )
        current_state = frontier.get()[1]
    return counter, len(current_state[0])

In [246]:
sum_counter_sol = 0
sum_counter_professor_sol = 0
steps = 3
for _ in tqdm(range(steps)):
    curr_sets = random_sets()
    c, len_sol = find_sol(curr_sets)
    sum_counter_sol += c
    c, len_professor_sol = find_sol_professor(curr_sets)
    sum_counter_professor_sol += c
    if len_sol != len_professor_sol:
        print("Solution is not optimal")
        print(f'sol: {len_sol}, professor_sol: {len_professor_sol}')
avg_sol = float(sum_counter_sol) / float(steps)
avg_professor_sol = float(sum_counter_professor_sol) / float(steps)
print(f'Average my solution steps: {avg_sol}')
print(f'Average professor solution steps: {avg_professor_sol}')

100%|██████████| 3/3 [00:16<00:00,  5.55s/it]

Average my solution steps: 100.0
Average professor solution steps: 84.0
